# We will use this notebook to create a final table, out of the three "clean" tables we have in DBeaver

In [ ]:
# import all libraries that are needed

import requests                 # requests on websites
from bs4 import BeautifulSoup   # html parsing

import pandas as pd             # pandas for data frame
import numpy as np              # numpy for array

import math                     # for math methods
import time                     # for sleep timer


from sql_functions import *     # functions from file for upload on schema
import psycopg2                 # for upload on engine

# Import the get_engine function from our sql_functions.
from sql_functions import get_engine #adjust this as necessary to match your sql_functions.py connection methods

# Import get_dataframe from sql_functions.py
from sql_functions import get_dataframe

# create a variable called engine using the get_engine function
engine = get_engine()

import datetime                 # for date and time
import datetime as dt           # for the csv file with the current date and time
from datetime import date       # for the current date

import re                       # for regular expressions

# https://jsonformatter.org/json-viewer
import json                     # for json files

import random                   # for random numbers

# https://jwalton.info/Matplotlib-custom-boxplots/
import itertools

import matplotlib.pyplot as plt # for plotting
# Will allow us to embed images in the notebook
%matplotlib inline
from matplotlib.cbook import _reshape_2D

# Seaborn visualization library
# https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166
import seaborn as sns

# Things we have to do in the final table, to get a decent database

In [ ]:
# we check the rows before dropping
plot_df.loc[(plot_df.property_type == 'Apartment')
                | ( plot_df.property_type == 'Flat' )
                | ( plot_df.property_type == 'Block of Apartments' )
                | ( plot_df.property_type == 'Flat' )
                | ( plot_df.property_type == 'Ground Flat' )
                | ( plot_df.property_type == 'Penthouse' )
                | ( plot_df.property_type == 'Studio') 
            ]
display(plot_df.head(2))

,platform_id,platform,neighbourhood,furniture,property_type,size_sqm,bedrooms,bathrooms,price,price_sqm,price_bedroom,available_from,available_today,let_type,scraping_date,price_band,price_sqm_band
0,127360856,Rightmove,Hammersmith and Fulham,unfurnished,Flat,NaN,2.0,2.0,3146.0,NaN,1573.0,None,unclear,Long term,2022-09-23,out of range,None
1,126259970,Rightmove,City of Westminster,furnished,Duplex,NaN,2.0,3.0,8017.0,NaN,4008.5,2022-10-01,occupied,Long term,2022-09-23,targeted price range,None


In [ ]:
# we drop our rows
df_short = plot_df.loc[(plot_df.property_type == 'Apartment')
                | ( plot_df.property_type == 'Flat' )
                | ( plot_df.property_type == 'Block of Apartments' )
                | ( plot_df.property_type == 'Flat' )
                | ( plot_df.property_type == 'Ground Flat' )
                | ( plot_df.property_type == 'Penthouse' )
                | ( plot_df.property_type == 'Studio') 
            ]

In [ ]:
df_short.property_type.unique()

array(['Flat', 'Studio', 'Apartment', 'Penthouse', 'Ground Flat',
       'Block of Apartments'], dtype=object)

In [ ]:
df_short.loc[
                        (
                            (df_short.price < 8001) & (df_short.furniture == 'furnished')
                         | ( df_short.furniture == 'unfurnished'
                            )
                            )]
display(df_short.head(2))

,platform_id,platform,neighbourhood,furniture,property_type,size_sqm,bedrooms,bathrooms,price,price_sqm,price_bedroom,available_from,available_today,let_type,scraping_date,price_band,price_sqm_band
0,127360856,Rightmove,Hammersmith and Fulham,unfurnished,Flat,NaN,2.0,2.0,3146.0,NaN,1573.0,None,unclear,Long term,2022-09-23,out of range,None
2,127210358,Rightmove,City of Westminster,furnished,Studio,NaN,0.0,1.0,2250.0,NaN,2250.0,None,unclear,Long term,2022-09-23,out of range,None


In [ ]:
df_short = df_short.loc[
                        (
                            (df_short.price < 8001) & (df_short.furniture == 'furnished')
                         | ( df_short.furniture == 'unfurnished'
                            )
                            )]

In [ ]:
df_short.head(2)

,platform_id,platform,neighbourhood,furniture,property_type,size_sqm,bedrooms,bathrooms,price,price_sqm,price_bedroom,available_from,available_today,let_type,scraping_date,price_band,price_sqm_band
0,127360856,Rightmove,Hammersmith and Fulham,unfurnished,Flat,NaN,2.0,2.0,3146.0,NaN,1573.0,None,unclear,Long term,2022-09-23,out of range,None
2,127210358,Rightmove,City of Westminster,furnished,Studio,NaN,0.0,1.0,2250.0,NaN,2250.0,None,unclear,Long term,2022-09-23,out of range,None


# Give the  data back to DBeaver to use it in Tableau

In [ ]:
# # Delete platforms_complete_4 table if it exists
sql_text = f"""
DROP TABLE IF EXISTS capstone_jmrs.platforms_complete_4;
    """ 
# Take the SQL-Command and execute it
result = engine.execute(sql_text)

In [ ]:
# # call the schema created for this project
schema = 'capstone_jmrs'

# give the table a unique name
table_name = 'platforms_complete_4'

# import the table to sql
if engine != None:
    try:
        df_short.to_sql(name=table_name,
                                   con=engine,
                                   if_exists='replace',
                                   schema=schema,
                                   index=False,
                                   chunksize=5000,
                                   method='multi')
        print(f"The {table_name} table was imported successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

# JULIA

### Merging the _clean tables

In [ ]:
sql_text = f"""
CREATE TABLE platforms_complete AS
(SELECT *
 FROM spotahome_clean
 UNION
 SELECT *
 FROM blueground_clean
 UNION
 SELECT *
 FROM rightmove_clean);
 """
result = engine.execute(sql_text)


### copy the 'old' table instead of overwriting

In [ ]:
sql_text = f"""
SELECT *
INTO platforms_complete_5
FROM platforms_complete_3;å
"""
result = engine.execute(sql_text)

### delete every property type that is not an Apartment, Flat, Studio etc

In [ ]:
sql_text = f"""
DELETE
FROM platforms_complete_5
WHERE property_type != 'Apartment'
	AND property_type != 'Block of Apartments'
	AND property_type != 'Flat'
	AND property_type != 'Ground Flat'
	AND property_type != 'Penthouse'
	AND property_type != 'Studio';
"""
result = engine.execute(sql_text)


### delete every property that is above £8000 for furnished apartments

In [ ]:
sql_text = f"""
DELETE
FROM platforms_complete_5
WHERE furniture = 'furnished'
	AND price > '8001';
"""
result = engine.execute(sql_text)

### delete every property that is above £xxxx for unfurnished apartments

In [ ]:
sql_text = f"""
DELETE
FROM platforms_complete_5
WHERE furniture = 'unfurnished'
	AND price > 'xxxx';
"""
result = engine.execute(sql_text)